In [1]:
import pandas as pd
import numpy as np
from library.utilities import check_missing

# Specify the path to your CSV file
file_path = '../Datasets/REMS_Mars_Dataset.csv'

# Load the CSV file into a Pandas DataFrame
mars_df = pd.read_csv(file_path)

# Display the first few rows to confirm successful import
print(mars_df.head())

         earth_date_time           mars_date_time sol_number  \
0  Earth, 2022-01-26 UTC  Mars, Month 6 - LS 163°   Sol 3368   
1  Earth, 2022-01-25 UTC  Mars, Month 6 - LS 163°   Sol 3367   
2  Earth, 2022-01-24 UTC  Mars, Month 6 - LS 162°   Sol 3366   
3  Earth, 2022-01-23 UTC  Mars, Month 6 - LS 162°   Sol 3365   
4  Earth, 2022-01-22 UTC  Mars, Month 6 - LS 161°   Sol 3364   

  max_ground_temp(°C) min_ground_temp(°C) max_air_temp(°C) min_air_temp(°C)  \
0                  -3                 -71               10              -84   
1                  -3                 -72               10              -87   
2                  -4                 -70                8              -81   
3                  -6                 -70                9              -91   
4                  -7                 -71                8              -92   

  mean_pressure(Pa)      wind_speed(m/h)          humidity(%) sunrise sunset  \
0               707  Value not available  Value not availabl

In [2]:
mars_df.describe()

,earth_date_time,mars_date_time,sol_number,max_ground_temp(°C),min_ground_temp(°C),max_air_temp(°C),min_air_temp(°C),mean_pressure(Pa),wind_speed(m/h),humidity(%),sunrise,sunset,UV_Radiation,weather
count,3197,3197,3197,3197,3197,3197,3197,3197,3197,3197,3197,3197,3170,3197
unique,3197,360,3197,52,40,51,78,224,1,1,90,96,4,2
top,"Earth, 2022-01-26 UTC","Mars, Month 3 - LS 86°",Sol 3368,-6,-79,14,-78,873,Value not available,Value not available,05:18,17:20,moderate,Sunny
freq,1,12,1,130,221,154,165,47,3197,3197,170,199,1450,3194


In [3]:
mars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3197 entries, 0 to 3196
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   earth_date_time      3197 non-null   object
 1   mars_date_time       3197 non-null   object
 2   sol_number           3197 non-null   object
 3   max_ground_temp(°C)  3197 non-null   object
 4   min_ground_temp(°C)  3197 non-null   object
 5   max_air_temp(°C)     3197 non-null   object
 6   min_air_temp(°C)     3197 non-null   object
 7   mean_pressure(Pa)    3197 non-null   object
 8   wind_speed(m/h)      3197 non-null   object
 9   humidity(%)          3197 non-null   object
 10  sunrise              3197 non-null   object
 11  sunset               3197 non-null   object
 12  UV_Radiation         3170 non-null   object
 13  weather              3197 non-null   object
dtypes: object(14)
memory usage: 349.8+ KB


In [4]:
missing = pd.concat([mars_df.isnull().sum(), 100 * mars_df.isnull().mean()], axis=1)
missing.columns = ['Count', '%']
missing = missing.sort_values(by='%', ascending=False)
print(missing)

                     Count         %
UV_Radiation            27  0.844542
earth_date_time          0  0.000000
mars_date_time           0  0.000000
sol_number               0  0.000000
max_ground_temp(°C)      0  0.000000
min_ground_temp(°C)      0  0.000000
max_air_temp(°C)         0  0.000000
min_air_temp(°C)         0  0.000000
mean_pressure(Pa)        0  0.000000
wind_speed(m/h)          0  0.000000
humidity(%)              0  0.000000
sunrise                  0  0.000000
sunset                   0  0.000000
weather                  0  0.000000


In [5]:
# Check for missing values and "Value not available"
missing_or_unavailable = pd.concat([
    mars_df.isnull().sum(),  # Count of NaN values
    mars_df.eq("Value not available").sum(),  # Count of "Value not available"
    100 * (mars_df.isnull().mean() + mars_df.eq("Value not available").mean())  # Percentage of missing or unavailable
], axis=1)

# Rename columns
missing_or_unavailable.columns = ['NaN Count', '"Value not available" Count', 'Total % Missing or Unavailable']

# Sort by percentage of missing or unavailable values
missing_or_unavailable = missing_or_unavailable.sort_values(by='Total % Missing or Unavailable', ascending=False)

# Display the result
print(missing_or_unavailable)


                     NaN Count  "Value not available" Count  \
wind_speed(m/h)              0                         3197   
humidity(%)                  0                         3197   
max_air_temp(°C)             0                           29   
min_air_temp(°C)             0                           29   
max_ground_temp(°C)          0                           28   
min_ground_temp(°C)          0                           28   
mean_pressure(Pa)            0                           27   
UV_Radiation                27                            0   
weather                      0                            3   
earth_date_time              0                            0   
mars_date_time               0                            0   
sol_number                   0                            0   
sunrise                      0                            0   
sunset                       0                            0   

                     Total % Missing or Unavailable  


In [6]:
# Check the number of unique values in a specific column
unique_values_count = mars_df.nunique()
print(f"Number of unique values in 'column_name': {unique_values_count}")


Number of unique values in 'column_name': earth_date_time        3197
mars_date_time          360
sol_number             3197
max_ground_temp(°C)      52
min_ground_temp(°C)      40
max_air_temp(°C)         51
min_air_temp(°C)         78
mean_pressure(Pa)       224
wind_speed(m/h)           1
humidity(%)               1
sunrise                  90
sunset                   96
UV_Radiation              4
weather                   2
dtype: int64


In [7]:
print("Unique UV Radiation Readings: ", mars_df['UV_Radiation'].unique())
print("Unique Weather Readings: ", mars_df['weather'].unique())
print("Unique Wind Speed Readings: ", mars_df['wind_speed(m/h)'].unique())
print("Unique Humidity Readings: ", mars_df['humidity(%)'].unique())

Unique UV Radiation Readings:  ['moderate' 'low' 'high' 'very_high' nan]
Unique Weather Readings:  ['Sunny' 'Value not available']
Unique Wind Speed Readings:  ['Value not available']
Unique Humidity Readings:  ['Value not available']


In [8]:
columns_to_drop = ['wind_speed(m/h)', 'humidity(%)', 'weather']
mars_df.drop(columns=columns_to_drop, inplace=True)

In [9]:
mars_df.head()

print(check_missing(mars_df))

                     NaN Count  "Value not available" Count  \
max_air_temp(°C)             0                           29   
min_air_temp(°C)             0                           29   
max_ground_temp(°C)          0                           28   
min_ground_temp(°C)          0                           28   
mean_pressure(Pa)            0                           27   
UV_Radiation                27                            0   
earth_date_time              0                            0   
mars_date_time               0                            0   
sol_number                   0                            0   
sunrise                      0                            0   
sunset                       0                            0   

                     Total % Missing or Unavailable  
max_air_temp(°C)                           0.907100  
min_air_temp(°C)                           0.907100  
max_ground_temp(°C)                        0.875821  
min_ground_temp(°C)        

In [10]:
# Reformat sol_number and set as index
mars_df['sol_number'] = mars_df['sol_number'].str.replace('sol', '', case=False).str.strip().astype(int)
print(mars_df['sol_number'].head())

0    3368
1    3367
2    3366
3    3365
4    3364
Name: sol_number, dtype: int32


In [11]:
# Set 'sol_number' as DataFrame index
mars_df.set_index('sol_number', inplace=True)

# Check DataFrame for new index
print(mars_df.head())

                  earth_date_time           mars_date_time  \
sol_number                                                   
3368        Earth, 2022-01-26 UTC  Mars, Month 6 - LS 163°   
3367        Earth, 2022-01-25 UTC  Mars, Month 6 - LS 163°   
3366        Earth, 2022-01-24 UTC  Mars, Month 6 - LS 162°   
3365        Earth, 2022-01-23 UTC  Mars, Month 6 - LS 162°   
3364        Earth, 2022-01-22 UTC  Mars, Month 6 - LS 161°   

           max_ground_temp(°C) min_ground_temp(°C) max_air_temp(°C)  \
sol_number                                                            
3368                        -3                 -71               10   
3367                        -3                 -72               10   
3366                        -4                 -70                8   
3365                        -6                 -70                9   
3364                        -7                 -71                8   

           min_air_temp(°C) mean_pressure(Pa) sunrise sunset UV_Rad

In [12]:
# Check for missing sol_numbers
missing_sols = mars_df.index.isnull().any()
if missing_sols:
    print("There are missing sol_numbers.")
else:
    print("No missing sol_numbers found.")


No missing sol_numbers found.


In [13]:
# Generate the full range of sol numbers
full_range = pd.RangeIndex(start=mars_df.index.min(), stop=mars_df.index.max() + 1)

# Check for missing sol numbers
missing_sols = full_range.difference(mars_df.index)
print("Missing sol numbers:", missing_sols.tolist())

# Check for repeated sol numbers
repeated_sols = mars_df.index[mars_df.index.duplicated()].unique()
print("Repeated sol numbers:", repeated_sols.tolist())


Missing sol numbers: [2, 3, 4, 5, 6, 7, 8, 193, 194, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 216, 217, 218, 219, 220, 221, 263, 264, 265, 266, 359, 360, 361, 362, 363, 364, 445, 446, 447, 448, 449, 450, 451, 452, 453, 458, 459, 460, 461, 479, 480, 481, 482, 483, 484, 873, 874, 875, 876, 877, 878, 879, 938, 955, 956, 999, 1027, 1289, 1290, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397, 1691, 1786, 2175, 2176, 2177, 2178, 2179, 2180, 2181, 2182, 2183, 2184, 2185, 2186, 2187, 2188, 2189, 2190, 2191, 2192, 2193, 2194, 2195, 2196, 2197, 2198, 2199, 2200, 2201, 2202, 2203, 2215, 2259, 2260, 2262, 2320, 2321, 2322, 2323, 2324, 2325, 2326, 2327, 2328, 2329, 2330, 2331, 2332, 2339, 2340, 2341, 2342, 2343, 2344, 2345, 2346, 2706, 2739, 2740, 2808, 2809, 2815, 2966, 3024, 3124, 3125, 3126, 3127, 3128, 3129, 3184, 3254, 3255, 3256, 3257, 3273, 3306, 3333, 3334, 3335, 3336, 3337, 3338, 3339, 3340, 3341, 3342, 3343, 3344, 3354]
Repeated sol numbers: []


In [14]:
# Reindex the DataFrame to include all sol numbers, filling gaps with NaN
mars_df = mars_df.reindex(full_range)
mars_df.replace("Value not available", np.nan, inplace=True) # Replace 'Value not available'

# Verify the new DataFrame
print("DataFrame after filling gaps with missing sol numbers:")
print(mars_df.head(15))  # Adjust the range as needed

DataFrame after filling gaps with missing sol numbers:
          earth_date_time           mars_date_time max_ground_temp(°C)  \
1   Earth, 2012-08-07 UTC  Mars, Month 6 - LS 150°                 NaN   
2                     NaN                      NaN                 NaN   
3                     NaN                      NaN                 NaN   
4                     NaN                      NaN                 NaN   
5                     NaN                      NaN                 NaN   
6                     NaN                      NaN                 NaN   
7                     NaN                      NaN                 NaN   
8                     NaN                      NaN                 NaN   
9   Earth, 2012-08-15 UTC  Mars, Month 6 - LS 155°                 NaN   
10  Earth, 2012-08-16 UTC  Mars, Month 6 - LS 155°                 -16   
11  Earth, 2012-08-17 UTC  Mars, Month 6 - LS 156°                 -11   
12  Earth, 2012-08-18 UTC  Mars, Month 6 - LS 156°       

In [15]:
print(check_missing(mars_df))

                     NaN Count  "Value not available" Count  \
max_air_temp(°C)           200                            0   
min_air_temp(°C)           200                            0   
max_ground_temp(°C)        199                            0   
min_ground_temp(°C)        199                            0   
mean_pressure(Pa)          198                            0   
UV_Radiation               198                            0   
earth_date_time            171                            0   
mars_date_time             171                            0   
sunrise                    171                            0   
sunset                     171                            0   

                     Total % Missing or Unavailable  
max_air_temp(°C)                           5.938242  
min_air_temp(°C)                           5.938242  
max_ground_temp(°C)                        5.908551  
min_ground_temp(°C)                        5.908551  
mean_pressure(Pa)                   

In [16]:
mars_df['earth_date_time'] = mars_df['earth_date_time'].str.replace('Earth, ', '', case=False).str.replace('UTC', '', case=False).str.strip()

In [17]:
mars_df['earth_date_time'] = pd.to_datetime(mars_df['earth_date_time'])
mars_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3368 entries, 1 to 3368
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   earth_date_time      3197 non-null   datetime64[ns]
 1   mars_date_time       3197 non-null   object        
 2   max_ground_temp(°C)  3169 non-null   object        
 3   min_ground_temp(°C)  3169 non-null   object        
 4   max_air_temp(°C)     3168 non-null   object        
 5   min_air_temp(°C)     3168 non-null   object        
 6   mean_pressure(Pa)    3170 non-null   object        
 7   sunrise              3197 non-null   object        
 8   sunset               3197 non-null   object        
 9   UV_Radiation         3170 non-null   object        
dtypes: datetime64[ns](1), object(9)
memory usage: 263.3+ KB


In [18]:
# Use a regular expression to extract the degree value from 'mars_date_time'
mars_df['mars_degree'] = mars_df['mars_date_time'].str.extract(r'LS\s+(\d+)°')[0].astype(float)

# Verify the new column
print(mars_df[['mars_date_time', 'mars_degree']].head())

columns = mars_df.columns.to_list()

columns.remove('mars_degree')
columns.insert(2, 'mars_degree')

mars_df = mars_df[columns]

# Drop the 'mars_date_time' column
mars_df.drop(columns=['mars_date_time'], inplace=True)

print(mars_df.head())

            mars_date_time  mars_degree
1  Mars, Month 6 - LS 150°        150.0
2                      NaN          NaN
3                      NaN          NaN
4                      NaN          NaN
5                      NaN          NaN
  earth_date_time  mars_degree max_ground_temp(°C) min_ground_temp(°C)  \
1      2012-08-07        150.0                 NaN                 NaN   
2             NaT          NaN                 NaN                 NaN   
3             NaT          NaN                 NaN                 NaN   
4             NaT          NaN                 NaN                 NaN   
5             NaT          NaN                 NaN                 NaN   

  max_air_temp(°C) min_air_temp(°C) mean_pressure(Pa) sunrise sunset  \
1              NaN              NaN               NaN   05:30  17:22   
2              NaN              NaN               NaN     NaN    NaN   
3              NaN              NaN               NaN     NaN    NaN   
4              NaN         

In [19]:
# Set temperatures to float
mars_df['max_ground_temp(°C)'] = mars_df['max_ground_temp(°C)'].astype(float) # Ground Temperatures
mars_df['min_ground_temp(°C)'] = mars_df['min_ground_temp(°C)'].astype(float)

mars_df['max_air_temp(°C)'] = mars_df['max_air_temp(°C)'].astype(float) # Air Temperatures
mars_df['min_air_temp(°C)'] = mars_df['min_air_temp(°C)'].astype(float)

# Set Pressure to Float
mars_df['mean_pressure(Pa)'] = mars_df['mean_pressure(Pa)'].astype(float)

# Set Sunrise/Sunset to Datetime
mars_df['sunrise'] = pd.to_datetime(mars_df['sunrise'], format='%H:%M').dt.time
mars_df['sunset'] = pd.to_datetime(mars_df['sunset'], format='%H:%M').dt.time

# Change UV_radiation to categorical data structure
# Create mapping for UV_Radiation levels
uv_mapping = {
    'low': 1,
    'moderate': 2,
    'high': 3,
    'very_high': 4
}

# Apply UV Mapping Schema
mars_df['UV_Radiation'] = mars_df['UV_Radiation'].map(uv_mapping).astype('Int64')
# Verify the transformation
print(mars_df['UV_Radiation'].unique())

# Impute missing Dates and Times

<IntegerArray>
[<NA>, 4, 3, 2, 1]
Length: 5, dtype: Int64
